In [ ]:
import os
import json
from urllib.parse import urlparse

collections_folder = './collections'

data = []

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

# List all files and directories in the specified folder
for root, dirs, files in os.walk(collections_folder):
    if root == collections_folder:
        continue

    collection_data = {}

    meta_file = os.path.join(root, 'meta.json')
    inscriptions_file = os.path.join(root, 'inscriptions.json')

    def read_json_file(file_path):
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                return json.load(f)
        return None

    collection_data['meta'] = read_json_file(meta_file)
    collection_data['data'] = read_json_file(inscriptions_file)

    if collection_data['data'] is None:
        print(f'No inscriptions found for {collection_data["meta"]["name"]}')
        continue

    if 'twitter_link' in collection_data['meta']:
        twitter_link = collection_data['meta']['twitter_link']
        if not is_valid_url(twitter_link):
            collection_data['meta']['twitter_link'] = f"https://twitter.com/{twitter_link}"

    if 'supply' in collection_data['meta']:
        del collection_data['meta']['supply']

    if 'discord_link' in collection_data['meta'] and not is_valid_url(collection_data['meta']['discord_link']):
        del collection_data['meta']['discord_link']

    if 'website_link' in collection_data['meta'] and not is_valid_url(collection_data['meta']['website_link']):
        del collection_data['meta']['website_link']

    data.append(collection_data)

In [ ]:
def remove_empty_strings(obj):
    if isinstance(obj, dict):
        return {k: remove_empty_strings(v) for k, v in obj.items() if v != ""}
    elif isinstance(obj, list):
        return [remove_empty_strings(v) for v in obj if v != ""]
    else:
        return obj
    

data = remove_empty_strings(data)

In [ ]:
from tqdm import tqdm
import requests
import pandas as pd

errors = []

cut = data
for i, collection in tqdm(enumerate(cut), total=len(cut)):
    url = 'http://prod-marketplace-backend:3000/admin/collection/onboard'
    d = json.dumps(collection)
    response = requests.post(url, data=d, headers={'Content-Type': 'application/json'})
    
    if response.status_code != 200:
        res = response.json()

        errors.append({
            'item': i,
            'collection': collection['meta']['name'],
            'slug': collection['meta']['slug'],
            'status': response.status_code,
            'response': res
        })

        if 'errors' in res:
            msg = res['errors'][0]['msg']
            if msg == 'collection already exists' or msg == 'some inscriptions already have a collection':
                continue

        print(f"item {i} failed with response {response.status_code} {response.text}")

print(f"failed items {len(errors)}")
errors_df = pd.DataFrame(errors)
errors_df.to_excel('onboard_errors.xlsx')